In [ ]:
!pip install -U "transformers" "huggingface_hub" "tokenizers"
!pip install Levenshtein
!pip install "protobuf==3.20.3" --force-reinstall
!pip install llm2vec


In [ ]:
import json
import pandas as pd
import numpy as np
import json
import pandas as pd
import numpy as np
import os, math, pandas as pd, numpy as np
from dataclasses import dataclass
from typing import Dict, List, Union, Optional
import torch
import Levenshtein
from typing import List, Sequence, Tuple
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer, DataCollatorWithPadding)
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import spearmanr, pearsonr
import editdistance
from peft import LoraConfig, get_peft_model, TaskType
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm import tqdm
import os
import random
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftModel
from collections import deque
from tqdm import tqdm
import os
import torch
import torch.optim as optim
import numpy as np
from transformers import get_scheduler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch.cuda.amp import autocast, GradScaler
import kagglehub
import random
from scipy.stats import spearmanr

from types import SimpleNamespace

In [ ]:
TASK_TYPE = "pair_cls"
MAX_LENGTH = 256
BATCH_SIZE = 4
epochs = 5
LR = 2e-5
warmup_ratio = 0.06
save_dir = 'ckpt1'
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

###
blending_model_special_token = "_" # teacher
base_model_special_token = "##" # student

In [ ]:


class TextPairRaw(Dataset):
    def __init__(self, df: pd.DataFrame, task: str):
        self.task = task
        if task == "single_cls":
            self.samples = [(t, None, int(y)) for t, y in zip(df["text"].astype(str), df["label"].astype(int))]
        elif task == "pair_cls":
            self.samples = [(a, b) for a,b in zip(df["premise"].astype(str),
                                                            df["hypothesis"].astype(str))]
        else:  # pair_reg
            self.samples = [(a, b) for a,b in zip(df["sentence1"].astype(str),
                                                              df["sentence2"].astype(str))]
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx] 
from typing import List, Tuple, Optional



class DualTokenizerCollate:
    def __init__(self, tok_student, tok_teacher, task: str, max_len: int):
        self.ts = tok_student
        self.tt = tok_teacher
        self.task = task
        self.max_len = max_len

    def __call__(self, batch: List[Tuple[str, Optional[str], float]]):
        s1s, s2s = zip(*batch)

        if self.task == "single_cls":
            s_enc = self.ts(list(s1s), max_length=self.max_len, truncation=True,
                            padding=True, return_tensors="pt",
                            return_special_tokens_mask=True)
            t_enc = self.tt(list(s1s), max_length=self.max_len, truncation=True,
                            padding=True, return_tensors="pt",
                            return_special_tokens_mask=True)

            out = {
                "input_ids_stu": s_enc["input_ids"],
                "attention_mask_stu": s_enc["attention_mask"],
                "special_tokens_mask_stu": s_enc["special_tokens_mask"],
                "input_ids_tea": t_enc["input_ids"],
                "attention_mask_tea": t_enc["attention_mask"],
                "special_tokens_mask_tea": t_enc["special_tokens_mask"],
                "labels": torch.tensor(ys, dtype=torch.long),
            }
            if "token_type_ids" in s_enc:
                out["token_type_ids_stu"] = s_enc["token_type_ids"]
            if "token_type_ids" in t_enc:
                out["token_type_ids_tea"] = t_enc["token_type_ids"]
            return out

        # ------- pair (bi-encoder) -------
        s1_enc = self.ts(list(s1s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)
        s2_enc = self.ts(list(s2s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)

        t1_enc = self.tt(list(s1s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)
        t2_enc = self.tt(list(s2s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)

        out = {
            # student
            "input_ids1_stu": s1_enc["input_ids"],
            "attention_mask1_stu": s1_enc["attention_mask"],
            "special_tokens_mask1_stu": s1_enc["special_tokens_mask"],
            "input_ids2_stu": s2_enc["input_ids"],
            "attention_mask2_stu": s2_enc["attention_mask"],
            "special_tokens_mask2_stu": s2_enc["special_tokens_mask"],
            # teacher
            "input_ids1_tea": t1_enc["input_ids"],
            "attention_mask1_tea": t1_enc["attention_mask"],
            "special_tokens_mask1_tea": t1_enc["special_tokens_mask"],
            "input_ids2_tea": t2_enc["input_ids"],
            "attention_mask2_tea": t2_enc["attention_mask"],
            "special_tokens_mask2_tea": t2_enc["special_tokens_mask"],
        }
        # chỉ thêm token_type_ids nếu tồn tại
        if "token_type_ids" in s1_enc:
            out["token_type_ids1_stu"] = s1_enc["token_type_ids"]
        if "token_type_ids" in s2_enc:
            out["token_type_ids2_stu"] = s2_enc["token_type_ids"]
        if "token_type_ids" in t1_enc:
            out["token_type_ids1_tea"] = t1_enc["token_type_ids"]
        if "token_type_ids" in t2_enc:
            out["token_type_ids2_tea"] = t2_enc["token_type_ids"]

        return out



# def smart_read(base, name_csv, name_tsv):
#     p_csv = os.path.join(base, name_csv)
#     p_tsv = os.path.join(base, name_tsv)
#     if os.path.exists(p_csv):
#         return pd.read_csv(p_csv)
#     if os.path.exists(p_tsv):
#         return pd.read_csv(p_tsv, sep="\t")
#     raise FileNotFoundError(f"Không thấy {p_csv} hoặc {p_tsv}")


# TASK_TYPE = "pair_cls"
# train_df = smart_read(BASE_INPUT, "train.csv", "train.tsv")
# dev_df   = smart_read(BASE_INPUT, "dev.csv",   "dev.tsv")
# test_df  = smart_read(BASE_INPUT, "test.csv",  "test.tsv")
# assert {"premise","hypothesis","label"}.issubset(train_df.columns), "SciTail cần premise,hypothesis"
print("Done Prepare Dataset")

In [ ]:
def info_nce(q, k, temperature=0.07, neg_valid_mask=None):
    q = F.normalize(q, dim=-1)
    k = F.normalize(k, dim=-1)

    logits = torch.matmul(q, k.T) / temperature
    labels = torch.arange(q.size(0), device=q.device)
    loss_inbatch = F.cross_entropy(logits, labels) 
    return loss_inbatch, logits

In [ ]:
data_train = '/kaggle/input/multitask-data/merged_9_data_3k_each_ver2.csv'
df = pd.read_csv(data_train)
df.head(3)

In [ ]:
df["premise"] = df["text"]
df["hypothesis"] = df["text"]
cols = ["premise", "hypothesis"]  # hoặc ["premise","hypothesis","label"] nếu cần placeholder
df_out = df[cols].copy()

In [ ]:
student_name = "huawei-noah/TinyBERT_General_4L_312D"
tok_student = AutoTokenizer.from_pretrained(student_name)
model_student = AutoModel.from_pretrained(
    student_name,
    output_hidden_states=True
)

tok_teacher = AutoTokenizer.from_pretrained("BAAI/bge-m3")
model_teacher = AutoModel.from_pretrained(
    "BAAI/bge-m3",
    output_hidden_states=True
)

In [ ]:

train_ds = TextPairRaw(df_out, TASK_TYPE)

collate = DualTokenizerCollate(tok_student, tok_teacher, TASK_TYPE, MAX_LENGTH)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=collate, pin_memory=True, num_workers=2, persistent_workers=True)
print("Done Prepare Dataloader")

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
from scipy.stats import pearsonr, spearmanr

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

class STSDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # instruction = "Given a text, Retrieve semantically similar text: "
        instruction=""
        return {
            "sentence1": instruction + self.dataset.iloc[idx]['sentence1'],
            "sentence2": instruction + self.dataset.iloc[idx]['sentence2'],
            "label": torch.tensor(self.dataset.iloc[idx]['score'], dtype=torch.float),
        }
        
def collate_fn(batch, tokenizer, max_len=128):
    s1_list = [item["sentence1"] for item in batch]
    s2_list = [item["sentence2"] for item in batch]
    labels = torch.stack([item["label"] for item in batch])

    enc1 = tokenizer(
        s1_list,
        truncation=True,
        padding=True,       # chỉ pad theo câu dài nhất trong batch
        max_length=max_len,
        return_tensors="pt"
    )
    enc2 = tokenizer(
        s2_list,
        truncation=True,
        padding=True,
        max_length=max_len,
        return_tensors="pt"
    )

    return {
        "input_ids1": enc1["input_ids"],
        "attention_mask1": enc1["attention_mask"],
        "input_ids2": enc2["input_ids"],
        "attention_mask2": enc2["attention_mask"],
        "labels": labels,
    }

def eval_sts(model, eval_loader):
    preds, labels = [], []
    device = model.device
    
    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                input_ids2 = batch["input_ids2"].to(device)
                attn2 = batch["attention_mask2"].to(device)
                label = batch["labels"]


                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                out2 = model(input_ids=input_ids2, attention_mask=attn2)

                # emb1 = mean_pooling(out1, attn1)
                # emb2 = mean_pooling(out2, attn2)
                emb1 = out1.last_hidden_state[:, 0, :]
                emb2 = out2.last_hidden_state[:, 0, :]
                # emb1 = out1.last_hidden_state[:, -1, :]
                # emb2 = out2.last_hidden_state[:, -1, :]
        
                # cosine similarity
                sim = F.cosine_similarity(emb1, emb2)
                score = (sim + 1) * 2.5  # scale [-1,1] -> [0,5]
        
                preds.extend(score.cpu().numpy())
                labels.extend(label.numpy())
    
    spearman_corr, _ = spearmanr(preds, labels)
    print(f"Spearman: {spearman_corr:.4f}")

    return spearman_corr


def eval_sts_task(model, path_list):
    model.eval()
    print(' ✅ eval_sts_task')
    for path in path_list:
        print(path)
        eval_dataset = STSDataset(path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: collate_fn(x, tokenizer)
        )
        eval_sts(model, eval_loader)
    model.train()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
import datasets
import numpy as np
import torch

def eval_cls(model, eval_loader):
    preds, labels = [], []
    device = model.device
    
    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                label = batch["labels"]

                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                emb1 = out1.last_hidden_state[:, 0, :]
        
                preds.extend(emb1.cpu().numpy())
                labels.extend(label.numpy())
    
    return preds, labels

class ClasssifyDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return {
            "text": self.dataset.iloc[idx]['text'],
            "label": torch.tensor(self.dataset.iloc[idx]['label'], dtype=torch.long),
        }

def clf_collate_fn(batch, tokenizer, max_len=512):
    s1_list = [item["text"] for item in batch]
    labels = torch.stack([item["label"] for item in batch])

    enc1 = tokenizer(
        s1_list,
        truncation=True,
        padding=True,       # chỉ pad theo câu dài nhất trong batch
        max_length=max_len,
        return_tensors="pt"
    )

    return {
        "input_ids1": enc1["input_ids"],
        "attention_mask1": enc1["attention_mask"],
        "labels": labels,
    }


def eval_classification_task(model, path_list):
    model.eval()
    print(' ✅ eval classifier')

    for train_path, dev_path in path_list:
        print(dev_path)
        eval_dataset = ClasssifyDataset(dev_path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: clf_collate_fn(x, tokenizer)
        )
        
        train_dataset = ClasssifyDataset(train_path)
        train_loader = DataLoader(
            train_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: clf_collate_fn(x, tokenizer)
        )

        X_train, y_train = eval_cls(model, train_loader)
        X_test, y_test = eval_cls(model, eval_loader)

        clf = LogisticRegression(
            random_state=42,
            n_jobs=1,
            max_iter=200,
            verbose=0,
        )
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        scores = {}
        accuracy = accuracy_score(y_test, y_pred)
        scores["accuracy"] = accuracy
        f1 = f1_score(y_test, y_pred, average="macro")
        scores["f1"] = f1
        print(scores)
        
    model.train()

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, average_precision_score


tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

class PairDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # instruction = "Given a text, Retrieve semantically similar text: "
        instruction=""
        return {
            "sentence1": instruction + self.dataset.iloc[idx]['sentence1'],
            "sentence2": instruction + self.dataset.iloc[idx]['sentence2'],
            "label": torch.tensor(self.dataset.iloc[idx]['label'], dtype=torch.float),
        }
        

def eval_pair(model, eval_loader):
    preds, labels = [], []
    device = model.device
    
    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                input_ids2 = batch["input_ids2"].to(device)
                attn2 = batch["attention_mask2"].to(device)
                label = batch["labels"]


                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                out2 = model(input_ids=input_ids2, attention_mask=attn2)

                emb1 = out1.last_hidden_state[:, 0, :]
                emb2 = out2.last_hidden_state[:, 0, :]
                # emb1 = out1.last_hidden_state[:, -1, :]
                # emb2 = out2.last_hidden_state[:, -1, :]
        
                # cosine similarity
                sim = F.cosine_similarity(emb1, emb2)
                score = (sim + 1) / 2
        
                preds.extend(score.cpu().numpy())
                labels.extend(label.numpy())
    
    metric = get_metric_pair_classification(preds, labels)
    print(metric)

    return metric

def get_metric_pair_classification(scores, labels):
    best_acc, best_thr = 0, 0
    for thr in np.linspace(0, 1, 200):
        preds = (scores >= thr).astype(int)
        acc = accuracy_score(labels, preds)
        if acc > best_acc:
            best_acc, best_thr = acc, thr
    preds = (scores >= best_thr).astype(int)
    return {
        "best_threshold": best_thr,
        "accuracy": best_acc,
        "f1": f1_score(labels, preds, average="macro"),
        "precision": precision_score(labels, preds, average="macro"),
        "recall": recall_score(labels, preds, average="macro"),
        "average_precision": average_precision_score(labels, scores)
    }


def eval_pair_task(model, path_list):
    model.eval()
    print(' ✅ eval_pair_task')
    for path in path_list:
        print(path)
        eval_dataset = PairDataset(path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: collate_fn(x, tokenizer)
        )
        eval_pair(model, eval_loader)
    model.train()

In [ ]:
eval_cls_tasks = [('/kaggle/input/multitask-data/multi-data/banking_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/banking77_validation.csv'),
                  ('/kaggle/input/multitask-data/multi-data/emotion_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/emotion_validation.csv'), 
                  ('/kaggle/input/multitask-data/multi-data/tweet_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/tweet_validation.csv')]

eval_sts_tasks = ['/kaggle/input/multitask-data/multi-data/sick_validation.csv', 
                  '/kaggle/input/multitask-data/multi-data/sts12_validation.csv', 
                  '/kaggle/input/multitask-data/multi-data/stsb_validation.csv']

eval_pair_tasks = ['/kaggle/input/multitask-data/multi-data/mrpc_validation.csv', 
                   '/kaggle/input/multitask-data/multi-data/scitail_validation.csv', 
                   '/kaggle/input/multitask-data/multi-data/wic_validation.csv']
test_cls_tasks = [('/kaggle/input/multitask-data/multi-data/banking_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/banking77_test.csv'),
                  ('/kaggle/input/multitask-data/multi-data/emotion_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/emotion_test.csv'), 
                  ('/kaggle/input/multitask-data/multi-data/tweet_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/tweet_test.csv')]

test_sts_tasks = ['/kaggle/input/multitask-data/multi-data/sick_test.csv', 
                  '/kaggle/input/multitask-data/multi-data/sts12_test.csv', 
                  '/kaggle/input/multitask-data/multi-data/stsb_test.csv']

test_pair_tasks = ['/kaggle/input/multitask-data/multi-data/mrpc_test.csv', 
                   '/kaggle/input/multitask-data/multi-data/scitail_test.csv', 
                   '/kaggle/input/multitask-data/multi-data/wic_test.csv']

# model_student.to("cuda")
# eval_classification_task(model_student, test_cls_tasks)
# eval_pair_task(model_student, test_pair_tasks)
# eval_sts_task(model_student, test_sts_tasks)

In [ ]:
from torch import Tensor
def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]


In [ ]:
best_f1_macro = -1.0
all_preds, all_labels = [], []
total_loss = 0.0

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


num_steps = len(train_loader)
total_traning_steps = num_steps * epochs
warmup_ratio = 0.1

def pick_devices():
    if torch.cuda.device_count() >= 2:
        dev_s = torch.device("cuda:0")  # student
        dev_t = torch.device("cuda:1")  # teacher
    elif torch.cuda.is_available():
        print("[WARN] Only 1 GPU available -> both on cuda:0")
        dev_s = dev_t = torch.device("cuda:0")
    else:
        print("[WARN] No GPU -> CPU")
        dev_s = dev_t = torch.device("cpu")
    return dev_s, dev_t

device_s, device_t = pick_devices()

model_student.to(device_s)
model_teacher.to(device_t)
model_teacher.eval()
for p in model_teacher.parameters():
    p.requires_grad_(False)

proj_s2t = None
proj_t2s = None

optimizer = optim.AdamW(model_student.parameters(), lr=LR)
scaler = GradScaler(enabled=torch.cuda.is_available())
scheduler = get_scheduler(
    name='cosine_with_min_lr',
    optimizer=optimizer,
    num_warmup_steps=int(total_traning_steps * warmup_ratio),
    # num_warmup_steps=1,
    num_training_steps=total_traning_steps,
    scheduler_specific_kwargs={'min_lr': 2e-6}
)
if save_dir:
    os.makedirs(save_dir, exist_ok=True)

n_items = 0
use_task_loss = True

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

def mean_pooling(last_hidden_state: torch.Tensor,
                 attention_mask: torch.Tensor) -> torch.Tensor:
    mask = attention_mask.unsqueeze(-1).type_as(last_hidden_state)
    summed = (last_hidden_state * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts

# --- devices (2x T4) ---
device_t = torch.device("cuda:0")
device_s = torch.device("cuda:1") if torch.cuda.device_count() > 1 else torch.device("cuda:0")

model_teacher = model_teacher.to(device_t).eval()
model_student = model_student.to(device_s)

use_amp = torch.cuda.is_available()
scaler = torch.amp.GradScaler("cuda", enabled=use_amp)

w_task     = 0.7
alpha_dtw  = 0.3
beta_score = 0.0
use_loss_tea = True

d_s = model_student.config.hidden_size
d_t = model_teacher.config.hidden_size

proj_s2t = nn.Linear(d_s, d_t, bias=False).to(device_s)
proj_t2s = nn.Linear(d_t, d_s, bias=False).to(device_s)

optimizer.add_param_group({"params": proj_s2t.parameters(), "lr": LR})
optimizer.add_param_group({"params": proj_t2s.parameters(), "lr": LR})

for epoch in range(epochs):
    model_student.train()
    total_loss, n_items = 0.0, 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

    for batch in pbar:
        batch_s, batch_t = {}, {}

        for k, v in batch.items():
            if not torch.is_tensor(v):
                continue

            # Student tensors -> device_s
            if k.endswith("_stu") or k == "labels":
                batch_s[k] = v.to(device_s, non_blocking=True)

            # Teacher tensors -> device_t  ✅ FIX
            if k.endswith("_tea"):
                batch_t[k] = v.to(device_t, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with torch.amp.autocast("cuda", enabled=use_amp):
            # ===== TEACHER on device_t =====
            with torch.no_grad():   # ✅ FIX: no_grad instead of inference_mode
                t_out1 = model_teacher(
                    input_ids=batch_t["input_ids1_tea"],
                    attention_mask=batch_t["attention_mask1_tea"],
                    output_hidden_states=True,
                    return_dict=True
                )
                t_out2 = model_teacher(
                    input_ids=batch_t["input_ids2_tea"],
                    attention_mask=batch_t["attention_mask2_tea"],
                    output_hidden_states=True,
                    return_dict=True
                )
        
                T_last1 = t_out1.last_hidden_state
                T_last2 = t_out2.last_hidden_state
                T_cls1  = mean_pooling(T_last1, batch_t["attention_mask1_tea"])
                T_cls2  = mean_pooling(T_last2, batch_t["attention_mask2_tea"])
        
            # Move teacher pooled vectors to student GPU (outside no_grad is fine too)
            T_cls1 = T_cls1.to(device_s, non_blocking=True)
            T_cls2 = T_cls2.to(device_s, non_blocking=True)


            # ===== STUDENT on device_s =====
            s_out1 = model_student(
                input_ids=batch_s["input_ids1_stu"],
                attention_mask=batch_s["attention_mask1_stu"],
                output_hidden_states=True, return_dict=True
            )
            s_out2 = model_student(
                input_ids=batch_s["input_ids2_stu"],
                attention_mask=batch_s["attention_mask2_stu"],
                output_hidden_states=True, return_dict=True
            )
            S_last1 = s_out1.last_hidden_state
            S_last2 = s_out2.last_hidden_state
            S_cls1  = S_last1[:, 0, :]
            S_cls2  = S_last2[:, 0, :]

            S_proj_cls1 = proj_s2t(S_cls1)     # [B, d_t]
            T_proj_cls1 = proj_t2s(T_cls1)     # [B, d_s]
            T_proj_cls2 = proj_t2s(T_cls2)     # [B, d_s]

            loss_task, _ = info_nce(S_cls1, S_cls2)
            loss_tea = F.mse_loss(T_proj_cls1, T_proj_cls2)

            align = S_proj_cls1 @ T_cls1.transpose(0, 1)     # [B,B]
            t2s_weight = torch.softmax(align, dim=-1)
            s2t_weight = torch.softmax(align.transpose(0, 1), dim=-1)

            T_to_S = proj_t2s(T_cls1)                         # [B, d_s]
            T2S_aligned = t2s_weight @ T_to_S                 # [B, d_s]
            S2T_aligned = s2t_weight @ S_proj_cls1            # [B, d_t]

            kd_t2s = F.mse_loss(S_cls1, T2S_aligned)
            kd_s2t = F.mse_loss(T_cls1.detach(), S2T_aligned)

            kd_sum = kd_t2s + kd_s2t + (loss_tea if use_loss_tea else 0.0)

            loss = w_task * loss_task + alpha_dtw * kd_sum
            loss = loss.float()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        bs = next(iter(batch_s.values())).size(0)
        total_loss += loss.item() * bs
        n_items += bs
        avg_loss = total_loss / max(1, n_items)
        pbar.set_postfix({"avg_loss": f"{avg_loss:.4f}"})

        # Optional: don't empty_cache every step; it's slow.
        del S_last1, S_last2, T_last1, T_last2, S_cls1, S_cls2, T_cls1, T_cls2, loss

    eval_classification_task(model_student, test_cls_tasks)
    eval_pair_task(model_student, test_pair_tasks)
    eval_sts_task(model_student, test_sts_tasks)


In [ ]:
# def mean_pooling(last_hidden_state: torch.Tensor,
#                  attention_mask: torch.Tensor) -> torch.Tensor:
#     mask = attention_mask.unsqueeze(-1).type_as(last_hidden_state)
#     # sum theo chiều L
#     summed = (last_hidden_state * mask).sum(dim=1)               # [B, d]
#     counts = mask.sum(dim=1).clamp(min=1e-9)                     # [B, 1]
#     return summed / counts

# w_task     = 0.7       # bi encoder
# alpha_dtw  = 0.3       # KD_DTW
# beta_score = 0.0       # mse matmul
# use_loss_tea = True

# d_s = model_student.config.hidden_size
# d_t = model_teacher.config.hidden_size

# proj_s2t = nn.Linear(d_s, d_t, bias=False).to(device_s)
# proj_t2s = nn.Linear(d_t, d_s, bias=False).to(device_s)

# optimizer.add_param_group({"params": proj_s2t.parameters(), "lr": LR})
# optimizer.add_param_group({"params": proj_t2s.parameters(), "lr": LR})

# for epoch in range(epochs):
#     model_student.train()
#     total_loss, n_items = 0.0, 0
#     pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

#     for batch in pbar:
#         batch_s, batch_t = {}, {}
#         for k, v in batch.items():
#             if not torch.is_tensor(v):
#                 continue
#             if k.endswith("_stu") or k == "labels":
#                 batch_s[k] = v.to(device_s, non_blocking=True)
#             if k.endswith("_tea"):
#                 batch_t[k] = v

#         optimizer.zero_grad(set_to_none=True)
        
#         with autocast(enabled=torch.cuda.is_available()):
#             # ========== TEACHER ==========
#             with torch.inference_mode():
#                 t_out1 = model_teacher(
#                     input_ids=batch_t["input_ids1_tea"],
#                     attention_mask=batch_t["attention_mask1_tea"],
#                     output_hidden_states=True,
#                     return_dict=True,
#                 )
#                 t_out2 = model_teacher(
#                     input_ids=batch_t["input_ids2_tea"],
#                     attention_mask=batch_t["attention_mask2_tea"],
#                     output_hidden_states=True,
#                     return_dict=True,
#                 )
        
#                 T_last1 = t_out1.last_hidden_state
#                 T_last2 = t_out2.last_hidden_state
        
#                 T_cls1 = mean_pooling(T_last1, batch_t["attention_mask1_tea"])
#                 T_cls2 = mean_pooling(T_last2, batch_t["attention_mask2_tea"])
        
#                 # move outputs to student device
#                 T_last1 = T_last1.to(device_s, non_blocking=True)
#                 T_last2 = T_last2.to(device_s, non_blocking=True)
#                 T_cls1  = T_cls1.to(device_s, non_blocking=True)
#                 T_cls2  = T_cls2.to(device_s, non_blocking=True)


#             # ========== STUDENT (bi-encoder) trên device_s ==========
#             s_out1 = model_student(
#                 input_ids=batch_s["input_ids1_stu"],
#                 attention_mask=batch_s["attention_mask1_stu"],
#                 output_hidden_states=True, return_dict=True
#             )
#             s_out2 = model_student(
#                 input_ids=batch_s["input_ids2_stu"],
#                 attention_mask=batch_s["attention_mask2_stu"],
#                 output_hidden_states=True, return_dict=True
#             )
#             S_last1 = s_out1.last_hidden_state        # [B, L1, d_s]
#             S_last2 = s_out2.last_hidden_state        # [B, L2, d_s]
#             S_cls1  = S_last1[:, 0, :]               # [B, d_s]
#             S_cls2  = S_last2[:, 0, :]

#             # if proj_s2t is None:
#             #     d_s, d_t = S_cls1.size(-1), T_cls1.size(-1)
#             #     proj_s2t = torch.nn.Linear(d_s, d_t, bias=False).to(device_s)
#             #     optimizer.add_param_group({"params": proj_s2t.parameters(), "lr": LR})

#             S_proj_cls1 = proj_s2t(S_cls1)          # [B, d_t]
#             # S_proj_cls2 = proj_s2t(S_cls2)          # [B, d_t]
#             # if proj_t2s is None:
#             #     d_s, d_t = S_cls1.size(-1), T_cls1.size(-1)
#             #     proj_t2s = torch.nn.Linear(d_t, d_s, bias=False).to(device_s)
#             #     optimizer.add_param_group({"params": proj_t2s.parameters(), "lr": LR})

#             T_proj_cls1 = proj_t2s(T_cls1)          # [B, d_s]
#             T_proj_cls2 = proj_t2s(T_cls2)          # [B, d_t]
#             base_dtype = S_last1.dtype
#             zero = torch.zeros((), device=device_s, dtype=base_dtype)

#             # ========== (A) HARD TASK LOSS: bi-encoder STS ==========
            
#             loss_task, _ = info_nce(S_cls1, S_cls2)
#             loss_tea = F.mse_loss(T_proj_cls1, T_proj_cls2)


#             # ========== (B) DSKD: align dim bằng proj, align L bằng weight align transpose ==========
#             kd_sum = 0.0
#             align = S_proj_cls1 @ T_cls1.transpose(0,1)
#             t2s_weight = torch.softmax(align, dim=-1)
#             s2t_weight = torch.softmax(align.transpose(0, 1), dim=-1)

            
#             T_to_S = proj_t2s(T_cls1) 
#             T2S_aligned = t2s_weight @ T_to_S
#             S2T_aligned = s2t_weight @ S_proj_cls1
#             kd_t2s = F.mse_loss(S_cls1, T2S_aligned)
#             T_cls1_new = T_cls1.detach().clone()
#             kd_s2t = F.mse_loss(T_cls1_new, S2T_aligned)
#             if use_loss_tea:
#                 kd_sum = kd_t2s + kd_s2t + loss_tea
#             else:
#                 kd_sum = kd_t2s + kd_s2t

#             # ========== (C) KD_SCORE: pairwise cosine matrix ==========
#             # t_norm = F.normalize(T_cls1, dim=-1).to(base_dtype)
#             # s_norm = F.normalize(S_proj_cls1, dim=-1)    
#             # teacher_scores = t_norm @ t_norm.transpose(0, 1)  
#             # student_scores = s_norm @ s_norm.transpose(0, 1)   
#             # kd_score = F.mse_loss(student_scores, teacher_scores)

#             # ========== TỔNG LOSS ==========
#             loss = w_task * loss_task + alpha_dtw * kd_sum
#             loss = loss.float()

#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
#         scheduler.step()

#         # ----- logging -----
#         bs = next(iter(batch_s.values())).size(0) 
#         total_loss += loss.item() * bs
#         n_items    += bs

#         mem_info = {}
#         for dev_id in range(torch.cuda.device_count()):
#             mem_alloc = torch.cuda.memory_allocated(dev_id) / 1024**2
#             mem_reserved = torch.cuda.memory_reserved(dev_id) / 1024**2
#             mem_info[f"gpu{dev_id}"] = f"{mem_alloc:.0f}/{mem_reserved:.0f}MB"

#         avg_loss = total_loss / max(1, n_items)
#         pbar.set_postfix({"avg_loss": f"{avg_loss:.4f}", **mem_info})

#         # dọn bộ nhớ
#         del S_last1, S_last2, T_last1, S_cls1, S_cls2, T_cls1, loss
#         torch.cuda.empty_cache()

#     eval_classification_task(model_student, test_cls_tasks)
#     eval_pair_task(model_student, test_pair_tasks)
#     eval_sts_task(model_student, test_sts_tasks)
